# Get Stock Data
Using yfinance api to get stock data in the form of Open, High, Low, Close price points of stock candlesticks

In [5]:
import pandas as pd
from pandas_datareader import data as web
import yfinance as yf
yf.pdr_override()
import numpy as np

STOCK = "ICICIBANK.NS"
START_DATE = "2021-05-05"
END_DATE = "2022-07-07"
STOCK_PATTERN = "HEAD_SHOULDERS"

df = web.get_data_yahoo(STOCK, start=START_DATE, end=END_DATE)
df['Id'] = np.arange(1, len(df)+1)
df.head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Id
Date,,,,,,,
2021-05-05,595.299988,604.799988,584.700012,602.799988,597.454773,17253943,1
2021-05-06,602.799988,610.849976,599.250000,608.200012,602.806946,16949763,2
2021-05-07,612.000000,614.799988,603.700012,607.099976,601.716675,10941083,3
2021-05-10,611.000000,617.000000,611.000000,613.250000,607.812134,12060808,4
2021-05-11,606.400024,612.900024,603.250000,609.849976,604.442261,13818480,5
2021-05-12,603.200012,606.650024,593.200012,594.750000,589.476196,14785569,6
2021-05-14,592.000000,601.950012,591.099976,597.299988,592.003540,12675754,7
2021-05-17,603.250000,624.500000,601.200012,623.599976,618.070312,20453101,8
2021-05-18,628.000000,635.500000,626.000000,633.200012,627.585266,19242627,9


# Get Strong and Weak Pivot points
Pivot points are points representing a particular candlestick which has High price point greater than NUM_BEFORE candles or NUM_AFTER candles. Same for the Low price points as well.

For strong pivot points - NUM_BEFORE and NUM_AFTER >= 10
For weak pivot points - NUM_BEFORE and NUM_AFTER >= 3

In [6]:
WEAK_NUM_BEFORE = 5
WEAK_NUM_AFTER = 5

STRONG_NUM_BEFORE = 15
STRONG_NUM_AFTER = 15

def pivotId(df, candle, num_before, num_after):
    if candle-num_before < 0 or candle+num_after >= len(df):
        return 0
    
    pivotIdLow=1
    pivotIdHigh=1
    for i in range(candle-num_before, candle+num_after):
        if(df.Low[candle]>df.Low[i]):
            pivotIdLow=0
        if(df.High[candle]<df.High[i]):
            pivotIdHigh=0
    if pivotIdLow and pivotIdHigh:
        return 3
    elif pivotIdLow:
        return 1
    elif pivotIdHigh:
        return 2
    else:
        return 0
    
df['Pivot'] = df.apply(lambda row: pivotId(df, int(row.Id)-1, WEAK_NUM_BEFORE, WEAK_NUM_AFTER), axis=1)
df['StrongPivot'] = df.apply(lambda row: pivotId(df, int(row.Id)-1, STRONG_NUM_BEFORE, STRONG_NUM_AFTER), axis=1)
df.tail(10)

,Open,High,Low,Close,Adj Close,Volume,Id,Pivot,StrongPivot
Date,,,,,,,,,
2022-06-23,690.200012,706.000000,687.000000,699.150024,694.979492,14960412,284,0,0
2022-06-24,710.900024,715.250000,706.549988,713.450012,709.194153,10856736,285,0,0
2022-06-27,728.849976,728.849976,716.099976,717.349976,713.070862,8285225,286,2,0
2022-06-28,711.049988,717.299988,707.000000,710.299988,706.062927,8339931,287,0,0
2022-06-29,704.950012,706.700012,700.950012,702.750000,698.557983,9570557,288,0,0
2022-06-30,702.750000,711.900024,701.200012,707.200012,702.981445,11970404,289,0,0
2022-07-01,703.450012,706.750000,694.099976,703.900024,699.701111,9884899,290,0,0
2022-07-04,704.849976,721.099976,704.849976,720.099976,715.804443,8303732,291,0,0
2022-07-05,723.049988,732.150024,718.099976,719.450012,715.158386,11460367,292,0,0


# Fix position of Pivot points
The position of pivot points is fixed just above or below a particular candle

In [7]:
def weakPointPosition(x):
    if x['Pivot']==1:
        return x['Low']-(0.01*df.High.max())
    elif x['Pivot']==2:
        return x['High']+(0.01*df.High.max())

def strongPointPosition(x):
    if x['StrongPivot']==1:
        return x['Low']-(0.03*df.High.max())
    elif x['StrongPivot']==2:
        return x['High']+(0.03*df.High.max())
    else:
        return np.nan

df['WeakPointPosition'] = df.apply(lambda row: weakPointPosition(row), axis=1)
df['StrongPointPosition'] = df.apply(lambda row: strongPointPosition(row), axis=1)
pd.set_option("display.max_rows", None, "display.max_columns", None)
display(df)

,Open,High,Low,Close,Adj Close,Volume,Id,Pivot,StrongPivot,WeakPointPosition,StrongPointPosition
Date,,,,,,,,,,,
2021-05-05,595.299988,604.799988,584.700012,602.799988,597.454773,17253943,1,0,0,NaN,NaN
2021-05-06,602.799988,610.849976,599.250000,608.200012,602.806946,16949763,2,0,0,NaN,NaN
2021-05-07,612.000000,614.799988,603.700012,607.099976,601.716675,10941083,3,0,0,NaN,NaN
2021-05-10,611.000000,617.000000,611.000000,613.250000,607.812134,12060808,4,0,0,NaN,NaN
2021-05-11,606.400024,612.900024,603.250000,609.849976,604.442261,13818480,5,0,0,NaN,NaN
2021-05-12,603.200012,606.650024,593.200012,594.750000,589.476196,14785569,6,0,0,NaN,NaN
2021-05-14,592.000000,601.950012,591.099976,597.299988,592.003540,12675754,7,1,0,582.429976,NaN
2021-05-17,603.250000,624.500000,601.200012,623.599976,618.070312,20453101,8,0,0,NaN,NaN
2021-05-18,628.000000,635.500000,626.000000,633.200012,627.585266,19242627,9,0,0,NaN,NaN


# Plot Pivot points

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.add_scatter(x=df.index, y=df['WeakPointPosition'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Weak Pivot")

fig.add_scatter(x=df.index, y=df['StrongPointPosition'], mode="markers",
                marker=dict(size=5, color="Brown"),
                name="Strong Pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Select Pivot points for Head and Shoulders Pattern
Points from recent date should be in order for Straight Head and Shoulders Pattern -
(min_pivot, max_pivot, min_pivot, max_short_pivot)